In [1]:
import findspark
findspark.init("/opt/spark")

In [12]:
from pyspark.sql.functions import explode, from_json, explode_outer
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import *

In [3]:
accessKeyId='dataops9'
secretAccessKey='Ankara06'

# create a SparkSession
spark = SparkSession.builder \
.appName("credits-silver") \
.master("local[2]") \
.config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.0,io.delta:delta-core_2.12:2.4.0") \
.config("fs.s3a.access.key", accessKeyId) \
.config("fs.s3a.secret.key", secretAccessKey) \
.config("fs.s3a.path.style.access", True) \
.config("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
.config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")\
.config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
.config("fs.s3a.endpoint", "http://minio:9000") \
.config("spark.sql.debug.maxToStringFields", 1000) \
.getOrCreate()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e3fcb6e3-008e-4965-ba41-697d9a569739;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.0 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.375 in central
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
downloading https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-aws/3.2.0/hadoop-aws-3.2.0.jar ...
	[SUCCESSFUL ] org.apache.hadoop#hadoop-aws;3.2.0!hadoop-aws.jar (329ms)
downloading https://repo1.maven.org/maven2/io/delta/delta-core_2.12/2.4.0/delta-core_2.12-2.4.0.jar ...
	[SUCCESSFUL ] io.delta#delta-core_2.12;2.4.0!delta-core_2.12.jar (2295ms)
downloading https://repo1.maven.org/maven2/com/amazonaws/aws-java

In [7]:
c_cast = DeltaTable.forPath(spark, "s3a://tmdb-silver/credits_cast")
c_cast = c_cast.toDF()

c_crew = DeltaTable.forPath(spark, "s3a://tmdb-silver/credits_crew")
c_crew = c_crew.toDF()

movies = DeltaTable.forPath(spark, "s3a://tmdb-silver/movies")
movies = movies.toDF()

genres = DeltaTable.forPath(spark, "s3a://tmdb-silver/genres")
genres = genres.toDF()

keywords = DeltaTable.forPath(spark, "s3a://tmdb-silver/keywords")
keywords = keywords.toDF()

production_companies = DeltaTable.forPath(spark, "s3a://tmdb-silver/production_companies")
production_companies = production_companies.toDF()

production_countries = DeltaTable.forPath(spark, "s3a://tmdb-silver/production_countries")
production_countries = production_countries.toDF()

spoken_languages = DeltaTable.forPath(spark, "s3a://tmdb-silver/spoken_languages")
spoken_languages = spoken_languages.toDF()

In [8]:
c_cast = c_cast.select("movie_id", "character", "name")
c_cast = c_cast.withColumnRenamed("movie_id", "cast_movie_id")

In [9]:
joined_df = movies.join(c_cast, movies.movie_id == c_cast.cast_movie_id)

In [10]:
joined_filt_tcruise = joined_df.filter(joined_df["name"] == 'Tom Cruise')

In [13]:
high_filt_tcruise = joined_filt_tcruise \
    .orderBy(desc("revenue"))\
    .select("title", "revenue", "character", "name") \
    .limit(1)
high_filt_tcruise.toPandas()

,title,revenue,character,name
0,Mission: Impossible - Ghost Protocol,694713380.0,Ethan Hunt,Tom Cruise


In [15]:
spoken_languages =spoken_languages.withColumnRenamed("movie_id", "spk_movie_id")
mv_spk_joined = movies.join(spoken_languages, movies.movie_id == spoken_languages.spk_movie_id) 

In [16]:
movie_counts = mv_spk_joined.groupBy("movie_id", "title") \
    .count()\
    .orderBy(desc("count"))\
    .limit(1)
movie_counts.show()

+--------+-----+-----+
|movie_id|title|count|
+--------+-----+-----+
|   14161| 2012|    9|
+--------+-----+-----+



In [18]:
mov_id = movie_counts.select("movie_id").collect()[0]["movie_id"]

most_repeat = mv_spk_joined\
    .filter(col("movie_id") == mov_id) \
    .select("title","iso_639_1", "name")

most_repeat.toPandas()

,title,iso_639_1,name
0,2012,zh,普通话
1,2012,ru,Pусский
2,2012,pt,Português
3,2012,la,Latin
4,2012,it,Italiano
5,2012,hi,हिन्दी
6,2012,fr,Français
7,2012,en,English
8,2012,bo,
